# Load Automated Annotations

In [1]:
 import os
import pandas as pd

# Dictionary to store DataFrames
dataframes = {}

# Root path where the results are stored
root_path = "../2_automated/results"

# Walk through each subfolder and file in the root path
for subdir, _, files in os.walk(root_path):
    for file in files:
        if file.endswith(".csv"):
            # Create a unique name for each DataFrame based on subfolder and file name
            relative_path = os.path.relpath(subdir, root_path)  # Get relative path for subfolder
            df_name = f"{relative_path}_{file.replace('.csv', '')}".replace(os.sep, "_")
            
            # Read the CSV file into a DataFrame
            file_path = os.path.join(subdir, file)
            dataframes[df_name] = pd.read_csv(file_path)
            
            print(f"Loaded {df_name} with shape {dataframes[df_name].shape}")

Loaded AffectNet - Test_face_analysis_results_anger with shape (1718, 5)
Loaded AffectNet - Test_face_analysis_results_contempt with shape (1312, 5)
Loaded AffectNet - Test_face_analysis_results_disgust with shape (1248, 5)
Loaded AffectNet - Test_face_analysis_results_fear with shape (1664, 5)
Loaded AffectNet - Test_face_analysis_results_happy with shape (2704, 5)
Loaded AffectNet - Test_face_analysis_results_neutral with shape (2368, 5)
Loaded AffectNet - Test_face_analysis_results_sad with shape (1584, 5)
Loaded AffectNet - Test_face_analysis_results_surprise with shape (1920, 5)
Loaded CK_face_analysis_results_anger_ck with shape (135, 5)
Loaded CK_face_analysis_results_contempt_ck with shape (54, 5)
Loaded CK_face_analysis_results_disgust_ck with shape (177, 5)
Loaded CK_face_analysis_results_fear_ck with shape (75, 5)
Loaded CK_face_analysis_results_happy_ck with shape (207, 5)
Loaded CK_face_analysis_results_sadness_ck with shape (84, 5)
Loaded CK_face_analysis_results_surprise

In [5]:
#dataframes["JAFFE_face_analysis_results__jaffe"]

# Merge

In [3]:
# Step 1: Create a list to hold modified DataFrames
modified_dfs = []

# Step 2: Process each DataFrame in the dictionary
for df_name, df in dataframes.items():
    # Split the 'Image' column into 'folderName' and 'imageName'
    split_image = df['Image'].str.split('_-_', n=1, expand=True)
    df['folderName'] = split_image[0]
    df['imageName'] = split_image[1].str.rsplit('.', n=1).str[0]  # Remove file extensions from imageName

    # Extract the dataset name from the folderName (first part of the folder name)
    df['datasetName'] = df_name.split("_")[0]  # Take the prefix as the dataset name
    
    # Append the modified DataFrame to the list
    modified_dfs.append(df)

# Step 3: Concatenate all modified DataFrames into a single DataFrame
combined_df = pd.concat(modified_dfs, ignore_index=True)


# Rearrange columns in the desired order
desired_column_order = ['datasetName', 'folderName', 'imageName', 'Age', 'Gender', 'Race']       
combined_df = combined_df[desired_column_order]
combined_df = combined_df.rename(columns={'imageName': 'Image'})


combined_file_path = 'AutomatedAnnotations_ALL.csv'
combined_df.to_csv(combined_file_path, index=False)

combined_df.head()

,datasetName,folderName,Image,Age,Gender,Race
0,AffectNet - Test,anger,image0000006,30,Woman,white
1,AffectNet - Test,anger,image0000060,34,Man,white
2,AffectNet - Test,anger,image0000061,44,Man,white
3,AffectNet - Test,anger,image0000066,49,Man,white
4,AffectNet - Test,anger,image0000106,30,Man,black


In [ ]:
# Replace NaN or None with 'Unknown' for all relevant columns
df['Final_Age'] = df['Final_Age'].fillna('Unknown')
df['Age'] = df['Age'].fillna('Unknown')
df['Final_Gender'] = df['Final_Gender'].fillna('Unknown')
df['Gender'] = df['Gender'].fillna('Unknown')
df['Final_Ethnicity'] = df['Final_Ethnicity'].fillna('Unknown')
df['Ethnicity'] = df['Ethnicity'].fillna('Unknown')


# Compute Cohen's Kappa for each attribute
age_kappa = cohen_kappa_score(df['Final_Age'], df['Age'])
gender_kappa = cohen_kappa_score(df['Final_Gender'], df['Gender'])
ethnicity_kappa = cohen_kappa_score(df['Final_Ethnicity'], df['Ethnicity'])

# Print results
print(f"Age Cohen's Kappa: {age_kappa:.2f}")
print(f"Gender Cohen's Kappa: {gender_kappa:.2f}")
print(f"Ethnicity Cohen's Kappa: {ethnicity_kappa:.2f}")


Age Cohen's Kappa: 0.23
Gender Cohen's Kappa: 0.40
Ethnicity Cohen's Kappa: 0.68


# EDA

In [ ]:
combined_df.shape

# 22890 all

### AffectNet Test: 14,518 images

diverse all age, gender and ethnicity




### FER2013 Test: 7,178 images - Black and white (grayscale) images only.


### CK (all images included): 981 images - Black and white (grayscale) images only.



### JAFFE (all images) 213 images

The JAFFE dataset was collected for controlled expression analysis rather than demographic diversity, so it lacks explicit demographic variability in age and ethnicity. This is a limitation when using JAFFE for tasks requiring diverse demographic information.

Age: The JAFFE dataset generally includes images of adult women, but specific ages are not provided within the dataset itself. It’s often assumed that the subjects are adults, typically ranging from young to middle-aged.

Gender: All individuals in the JAFFE dataset are female, as the dataset name itself specifies ("Japanese Female Facial Expression").

Ethnicity: All subjects in the JAFFE dataset are of Japanese ethnicity, making the dataset homogeneous in terms of ethnicity.

### Why Grayscale Images Are Unsuitable for Age, Gender, and Ethnicity Annotation

Annotating age, gender, and ethnicity from grayscale (black and white) images is challenging and often unreliable due to the lack of color information. Color plays a crucial role in accurately assessing demographic attributes:

Age Estimation: Age-related features like skin texture, wrinkles, and hair color are harder to discern without color. Grayscale images obscure these details, making it difficult to differentiate between similar age groups accurately.

Gender Identification: Indicators such as makeup, lip color, and hair color, which can subtly signal gender, are lost in grayscale, reducing the accuracy of gender annotations.

Ethnicity Identification: Ethnicity often relies on skin tone and undertones, which are absent in grayscale images. This makes it nearly impossible to distinguish between certain ethnic groups, leading to unreliable results.

In summary, grayscale images lack the necessary detail to make accurate annotations of age, gender, and ethnicity, and attempting to do so may introduce significant bias and error into the dataset.